In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = '../input/train-rg-transformed/train_50.csv'
test_path =  '../input/train-rg-transformed/test_50.csv'
val_path = '../input/train-rg-transformed/val_50.csv'

In [8]:
oof_train_path = 50
oof_test_path = 50

In [9]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 2.5 s, sys: 138 ms, total: 2.64 s
Wall time: 3.17 s


In [10]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [11]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 4200), (804, 4200))

In [12]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [13]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [14]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [15]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [16]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [17]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [18]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [19]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [20]:
train_set_100.shape, test_set_100.shape

((1059, 4200), (804, 4200))

In [21]:
assert (train_set_100.columns == test_set_100.columns).all()

In [22]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 166.92it/s]


(1059, 101)


100%|██████████| 100/100 [00:01<00:00, 61.04it/s]


(1059, 4100)


100%|██████████| 100/100 [00:01<00:00, 56.45it/s]

(1059, 4400)


In [23]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 142.04it/s]


(804, 101)


100%|██████████| 100/100 [00:01<00:00, 63.00it/s]


(804, 4100)


100%|██████████| 100/100 [00:01<00:00, 70.69it/s]

(804, 4400)


In [24]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [25]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [26]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [27]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [28]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],1)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.3112205505177341


100%|██████████| 101/101 [00:00<00:00, 312177.38it/s]

13


0.25889590246973515


100%|██████████| 4100/4100 [00:00<00:00, 842436.02it/s]


...

4995:	learn: 0.0039054	total: 1m 50s	remaining: 88.2ms
4999:	learn: 0.0039026	total: 1m 50s	remaining: 0us
fold Loss 17:  0.08394958272075288
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6882109	total: 52.8ms	remaining: 4m 24s
999:	learn: 0.0331370	total: 21.9s	remaining: 1m 27s
1998:	learn: 0.0137938	total: 44s	remaining: 1m 6s
2997:	learn: 0.0078881	total: 1m 6s	remaining: 44.5s
3996:	learn: 0.0049440	total: 1m 28s	remaining: 22.2s
4995:	learn: 0.0033982	total: 1m 51s	remaining: 89ms
4999:	learn: 0.0033922	total: 1m 51s	remaining: 0us
fold Loss 18:  0.0742055933105827


19it [35:34, 112.16s/it]

---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6884790	total: 35.9ms	remaining: 2m 59s
999:	learn: 0.0302906	total: 22.1s	remaining: 1m 28s
1998:	learn: 0.0124227	total: 43.9s	remaining: 1m 5s
2997:	learn: 0.0069461	total: 1m 5s	remaining: 44s
3996:	learn: 0.0043679	total: 1m 28s	remaining: 22.1s


20it [37:24, 112.24s/it]

4995:	learn: 0.0031260	total: 1m 49s	remaining: 87.7ms
4999:	learn: 0.0031203	total: 1m 49s	remaining: 0us
fold Loss 19:  0.12189915603979418
---------------------------------------------------------------------------
Training Loss: 0.008957037941635786
Loss for sulfide:
0.06697304900080507 0.051164679990135783
AGG Loss: [0.13523814 0.09320002 0.1451895  0.18776562 0.00345388 0.13894088
 0.19227263 0.16717187 0.15894448 0.06697305]
AGG Loss mean: 0.12891500581006143


In [29]:
print(np.array(agg_loss).mean())

0.12891500581006143


In [30]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [31]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.004920,0.002571,0.010413,0.012862,0.000117,0.007314,0.008934,0.075656,0.004939,0.000899
767,NaN,0.029149,0.010581,0.002818,0.024433,0.000032,0.012178,0.009938,0.086139,0.024871,0.000440
768,NaN,0.864638,0.095192,0.001638,0.028180,0.000122,0.009072,0.803618,0.853868,0.027590,0.000679
769,NaN,0.009222,0.006101,0.040915,0.024280,0.000054,0.947334,0.036886,0.009516,0.972832,0.000486
770,NaN,0.007906,0.004947,0.018391,0.773493,0.000040,0.992211,0.932514,0.004738,0.008075,0.000250
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.005651,0.002511,0.031462,0.008105,0.000059,0.005049,0.011490,0.066107,0.011144,0.000468
1055,NaN,0.009371,0.006297,0.011407,0.034309,0.000333,0.011191,0.011377,0.901242,0.004724,0.000667
1056,NaN,0.003321,0.006934,0.003141,0.011838,0.998765,0.001932,0.005337,0.002906,0.005435,0.000776
1057,NaN,0.007976,0.926821,0.012720,0.053598,0.000143,0.006770,0.031534,0.011801,0.009296,0.000764


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.005163,0.002943,0.014436,0.017151,0.000123,0.008024,0.010461,0.287760,0.005197,0.001178
767,NaN,0.063424,0.005208,0.003448,0.045682,0.000029,0.037232,0.022854,0.374969,0.067271,0.000404
768,NaN,0.558726,0.673489,0.001816,0.056479,0.000164,0.010226,0.395195,0.456120,0.080964,0.000616
769,NaN,0.019506,0.008379,0.127465,0.027055,0.000057,0.831829,0.069445,0.009443,0.958582,0.000585
770,NaN,0.006901,0.006396,0.021684,0.380723,0.000043,0.992322,0.872331,0.005558,0.008566,0.000295
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.008321,0.001495,0.030932,0.008376,0.000067,0.005874,0.017698,0.225176,0.014890,0.000426
1055,NaN,0.012861,0.007107,0.010807,0.056974,0.000293,0.014752,0.016776,0.458400,0.005234,0.000655
1056,NaN,0.003629,0.007813,0.003831,0.012210,0.998525,0.002380,0.004780,0.003441,0.007202,0.000730
1057,NaN,0.010630,0.849320,0.013723,0.090311,0.000133,0.012906,0.050086,0.015510,0.011215,0.000736


In [32]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.13569630721695858
carbonate 0.08448268657585337
chloride 0.1542354673731409
iron_oxide 0.24205443372939314
oxalate 0.0009891664319531459
oxychlorine 0.1287295740075028
phyllosilicate 0.171787905903117
silicate 0.14657444138269937
sulfate 0.18344141847233975
sulfide 0.06871408527070473
0.13167054863636624


In [33]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.026584023721495584
carbonate 0.015247477307117297
chloride 0.0326664523366399
iron_oxide 0.05639170676640333
oxalate 0.00036401475028529913
oxychlorine 0.023879557563627973
phyllosilicate 0.04544378836400112
silicate 0.03319567960095909
sulfate 0.029122409542294253
sulfide 0.0048059433600600165
0.026770105331288385


In [34]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [35]:
oof_train_df.to_csv('oof_train_50_20fold_5kest.csv', index=False)
oof_test_df.to_csv('oof_test_50_20fold_5kest.csv', index=False)